#### Python library for the analysis of raw data from the Consumer Physics SCIO


In [24]:
import json
import struct
import logging
import datetime
import os
import csv
import base64
import serial
import serial.tools.list_ports as pyserial
import numpy as np

# Logging/output format setup
log = logging.getLogger('root')
log.setLevel(logging.DEBUG)
logging.basicConfig(format='[%(asctime)s] %(levelname)8s: %(message)s', datefmt='%Y-%m-%d %H:%M:%S')

In [25]:
class scio_analyse:
    def __init__(self):
        self.device_info = {}
        self.scan_rawdata = {}
        self.calibration_data = {}
        self.key = b''
    
    def load_scio_scan(self):
        return
    
    def parse_scan(self):
        # Convert to bytes
        sample_bytes          = base64.b64decode(self.scan_rawdata['sample']) 
        sample_dark_bytes     = base64.b64decode(self.scan_rawdata['sample_dark'])
        sample_gradient_bytes = base64.b64decode(self.scan_rawdata['sample_gradient'])
        
        # Same for calibration
        sample_cal_bytes          = base64.b64decode(self.calibration_data['sample']) 
        sample_dark_cal_bytes     = base64.b64decode(self.calibration_data['sample_dark'])
        sample_gradient_cal_bytes = base64.b64decode(self.calibration_data['sample_gradient'])
        
        # Decode
        sample_raw          = np.array(struct.unpack('>4x400I196x', sample_bytes))
        sample_dark_raw     = np.array(struct.unpack('>4x400I196x', sample_dark_bytes))
        sample_gradient_raw = np.array(struct.unpack('>4x400I52x',  sample_gradient_bytes))
        sample_cal          = np.array(struct.unpack('>4x400I196x', sample_cal_bytes))
        sample_dark_cal     = np.array(struct.unpack('>4x400I196x', sample_dark_cal_bytes))
        sample_gradient_cal = np.array(struct.unpack('>4x400I52x',  sample_gradient_cal_bytes))
        
        # Calibration
        sample          = sample_raw - sample_cal
        sample_dark     = sample_dark_raw - sample_dark_cal
        sample_gradient = sample_gradient_raw - sample_gradient_cal
        
        # Calculate reflectance
        corrected_sample   = sample - sample_dark
        corrected_gradient = sample_gradient - sample_dark
        reflectance        = corrected_sample / corrected_gradient
        
        return(reflectance)
    
    def load_calibration_file(self, calibration_fn = 'calibration.json'):
        # Load the JSON file
        with open(calibration_fn, 'r') as file:
            json_data = json.load(file)

        # Load the "scan" object
        self.calibration_data = json_data['scan']
        
        # Load the device information object
        self.device_info = json_data['device']
        
        return(self.calibration_data)
    
    # Saves a single scan as raw data to JSON
    def write_data_file(self, output_fn, scan_rawdata, temp_before, temp_after):
        timestamp = datetime.datetime.utcnow().strftime('%Y-%m-%d %H:%M:%S')
        
        # Rename temperature reading dicts
        temp_before['cmos_t_before'] = temp_before.pop('cmos_t')
        temp_before['chip_t_before'] = temp_before.pop('chip_t')
        temp_before['obj_t_before']  = temp_before.pop('obj_t')
        temp_after['cmos_t_after']   = temp_after.pop('cmos_t')
        temp_after['chip_t_after']   = temp_after.pop('chip_t')
        temp_after['obj_t_after']    = temp_after.pop('obj_t')
        
        # Create the json string
        jsondata = {}
        jsondata['device'] = self.device_info
        jsondata['scan'] = {'timestamp': timestamp}
        jsondata['scan'].update(temp_before)
        jsondata['scan'].update(temp_after)
        jsondata['scan'].update(scan_rawdata)
        # Write data to JSON file
        with open(output_fn, 'w') as outfile:
            json.dump(jsondata, outfile, indent=4)
        return
    
    def encode_b64(self, bytestring):
        urlSafeEncodedBytes = base64.urlsafe_b64encode(bytestring)
        urlSafeEncodedStr = str(urlSafeEncodedBytes, 'utf-8')
        return(urlSafeEncodedStr)
    
    def run_parsing(self, data_fn, calibration_fn, silent=True):
        return
    
    # Generic, for testing
    def test_parse_response(self, response_data_raw):
        # Parse response
        print('As hex:', response_data_raw[2].hex())
        print(len(response_data_raw[2]))
        
        data_struct = '<4I' # File list
        resp = list(struct.unpack(data_struct, response_data_raw[2]))
        print(resp)
        file_list = {}
        for file_nb in range(0, int(len(resp)/2)):
            if((resp[file_nb*2] >= 87) & (resp[file_nb*2] <= 95)):
                file_list.update({file_nb:{'file_type':resp[file_nb*2], 'file_version':resp[file_nb*2+1]}})
                key_start_byte = file_nb*8
                key_end_byte   = file_nb*8+4
                val_start_byte = file_nb*8+4
                val_end_byte   = file_nb*8+8
                attr = {'key':response_data_raw[2][key_start_byte:key_end_byte].hex(),
                        'value':response_data_raw[2][val_start_byte:val_end_byte].hex()}
                file_list.update({file_nb:attr})
            else:
                attr = {'file_type':resp[file_nb*2], 'file_version':resp[file_nb*2+1]}
                file_list.update({file_nb:attr})
            pass
        
        return(resp)

In [26]:
calibration_fn = './calibration.json'
data_fn        = './scan.json'

# Now set up device
scio = scio_analyse() # Create an instance for this device address
scio.load_calibration_file()
print(scio.calibration_data)
scio.run_parsing(data_fn, calibration_fn, silent=False)
print('Done...')

{'timestamp': '2023-03-19 13:37:29', 't_cmos_before': 16.874822594379772, 't_chip_before': 22.87, 't_obj_before': 0.0, 't_cmos_after': 16.874822594379772, 't_chip_after': 23.37, 't_obj_after': 0.0, 'sample': 'AAAAAFnkHt3y1iBRQgFUpgblQwdUH4yoyGAPy2WYmSsPQRBvd3Lb88tZKfhG7VEpHmwHbYqocLVUFJRPmY4MkZ5NKLgaVpuXOYAmw20-hShIDEkWM6jRDzUzNzJeNg3LWIADQK462RMw9md16jkv34TWIz59k80ifdbs772DUkEE0cy5wShonw7hsHPX3t9RTnbsam44fjJBUlG_MKtCdLGg6GXIJ7SBKV9sPn7ADafgiUnsoMPlmEuSlISQcIT_6DPVyUiyXL-Xg6XSc2OTAwHCRzUcqRBCvsd_NVDDr17yXeD33KAAvNKhb3Jua21bfvafrOglbnwJX8g1SqTgL-BQ9hgcgtenG5lat5yR6IeOZQ8mgZQBvPERIzLlE-I7w182L9NghvKfy2dmFeujgi5nILTNYi_HUnJ8wX-7bZEiiO1Mn0M7maOuB0_4fdjC5qWtohWpzpq06EL0IU4YoeApWFY6u0ujjGltn60XgYbappUxTXaIufFb9Q5XTaxNMkyU6x2Bl0S5SEHAwV201nPWiHbJIZ-6MWqnLd3FFpgtn1JchisYiD4vmOQKvFP7UefX9YK-Xyu0eg9-N-AmR8GWwk7rmCslGB51wFscBQ3FNfL-wnI7ZHJJkFsok1NEu2ZHBGI2GaoHrCjOvNvhnS3DulGxO4jUxBTwVMgl9Tnvf43xfjPm5pM5OmULBbdXTFwe34uiP2nFbfr-2AoxVa7o7mjYYbBQw_gzR9y6a-ezlltiMdsFOpM5pM-C8s9GnhIa54gqjCeRmJOZUaRCFz-